<a href="https://colab.research.google.com/github/Nguyencongdat1997/A.n.D/blob/master/ver1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import

In [50]:
# example of loading the mnist dataset
from keras.datasets import mnist
from matplotlib import pyplot
# load dataset
(trainX, trainy), (testX, testy) = mnist.load_data()
# summarize loaded dataset
print('Train: X=%s, y=%s' % (trainX.shape, trainy.shape))
print('Test: X=%s, y=%s' % (testX.shape, testy.shape))
# plot first few images
# for i in range(9):
# 	# define subplot
# 	pyplot.subplot(330 + 1 + i)
# 	# plot raw pixel data
# 	pyplot.imshow(trainX[i], cmap=pyplot.get_cmap('gray'))
# # show the figure
# pyplot.show()

Train: X=(60000, 28, 28), y=(60000,)
Test: X=(10000, 28, 28), y=(10000,)


In [12]:
#https://github.com/enggen/Deep-Learning-Coursera/blob/master/Convolutional%20Neural%20Networks/Week1/Convolution%20model%20-%20Step%20by%20Step%20-%20v2.ipynb

import numpy as np
import matplotlib.pyplot as plt
import time

np.random.seed(1)

#Layers

In [28]:
# def sigmoid(Z):
#     A = 1/(1+np.exp(-Z))
#     cache = Z
#     return A, cache

# def softmax(self,z):
#     # implement the softmax function
#     return 1/sum(np.exp(z)) * np.exp(z)

def sigmoid(x, derivative=False):
    if derivative:
        return (np.exp(-x))/((np.exp(-x)+1)**2)
    return 1/(1 + np.exp(-x))

def softmax(x, derivative=False):
    # Numerically stable with large exponentials
    exps = np.exp(x - x.max())
    if derivative:
        return exps / np.sum(exps, axis=0) * (1 - exps / np.sum(exps, axis=0))
    return exps / np.sum(exps, axis=0)

def relu(Z):
    A = np.maximum(0,Z)
    cache = Z 
    return A, cache

def tanh(Z):
    A = (np.exp(Z)-np.exp(-Z))/(np.exp(Z)+np.exp(-Z))
    cache = Z 
    return A, cache

def relu_backward(dA, Z):
    dZ = np.array(dA, copy=True) # convert dz to a correct object.    
    dZ[Z <= 0] = 0 # When z <= 0, should set dz to 0 . 
    return dZ

def sigmoid_backward(dA, Z):
    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)
    return dZ

def tanh_backward(dA, Z):
    a, cache = tanh(Z)
    dZ = dA * (1-a**2)
    return dZ

class LinearLayer:
    def __init__(self, W, b, activation="relu"):
        self.W= W
        #self.b= b
        
        self.stored_A_prev = None
        self.stored_A = None
        self.stored_Z = None
        
        self.type = "linear"
        self.activation = activation

    def udpate_weight(self, dW, db, learning_rate):
        self.W = self.W - learning_rate * dW
        #self.b = self.W - learning_rate * db

    def forward(self, A_prev, remember_for_backprop=True):
        Z = np.dot(self.W, A_prev) # + b
        if self.activation == "sigmoid":
            A = sigmoid(Z)
        elif self.activation == "relu":
            A = relu(Z)
        elif self.activation == "softmax":
            A = softmax(Z)
        
        self.stored_A_prev = A_prev
        self.stored_Z = Z
        self.stored_A = A

        return A

    def backward(self, error):   
        '''
        neu softmax: error = 2 * (output - y_train) / output.shape[0]
        neu sigmoid: error = np.dot(params['W_l+1'].T, error) *

        '''
        if self.activation == "sigmoid":
            error = error* sigmoid(self.stored_Z, derivative=True)
            change_w = np.outer(error, self.stored_A_prev)
        elif self.activation == "relu":
            pass
        elif self.activation == "softmax":
            error = error* sigmoid(self.stored_Z, derivative=True)
            change_w = np.outer(error, self.stored_A_prev)
        return error, change_w
    

#Model

In [30]:
class Model:
    def __init__(self, x=784, h1=300, h2=300 ,y=10):
        W1,b1 =  self.initialize_linear_param(x, h1)
        self.layer1 = LinearLayer(W1, b1, activation="sigmoid")  
        W2,b2 =  self.initialize_linear_param(h1, h2)
        self.layer2 = LinearLayer(W2, b2, activation="sigmoid") 
        W3,b3 =  self.initialize_linear_param(h2, y)
        self.layer3 = LinearLayer(W3, b3, activation="softmax")
        

    def L_model_forward(self, X):
        # input layer activations becomes sample
        a0 = X.reshape((784,1))

        # input layer to hidden layer 1
        # params['Z1'] = np.dot(params["W1"], params['A0'])
        # params['A1'] = self.sigmoid(params['Z1'])
        a1 = self.layer1.forward(a0)

        # hidden layer 1 to hidden layer 2
        # params['Z2'] = np.dot(params["W2"], params['A1'])
        # params['A2'] = self.sigmoid(params['Z2'])
        a2 = self.layer2.forward(a1)

        # hidden layer 2 to output layer
        # params['Z3'] = np.dot(params["W3"], params['A2'])
        # params['A3'] = self.softmax(params['Z3'])
        a3 = self.layer3.forward(a2)

        return a3      

    def initialize_linear_param(self, n_x, n_y):
        W = np.random.randn(n_y, n_x) * 0.01
        b = np.zeros(shape=(n_y, 1))
        return W,b 

    def L_model_backward(self, y_train, output):
        change_w = {}
        # Calculate W3 update
        error = 2 * (output - y_train) / output.shape[0]
        error, change_w['W3'] = self.layer3.backward(error)

        # Calculate W2 update
        error = np.dot(self.layer3.W.T, error) 
        error, change_w['W2'] = self.layer2.backward(error)

        # Calculate W1 update
        error = np.dot(self.layer2.W.T, error) 
        error, change_w['W1'] = self.layer1.backward(error)

        return change_w

    def update_network_parameters(self, change_w, lr):
        db = 0
        self.layer1.udpate_weight(change_w['W1'], db, lr)
        self.layer2.udpate_weight(change_w['W2'], db, lr)
        self.layer3.udpate_weight(change_w['W3'], db, lr)

#Trainer

In [51]:
def compute_accuracy(model, x_val, y_val):
        predictions = []
        for x, y in zip(x_val, y_val):
            output = model.L_model_forward(x)
            pred = np.argmax(output)
            predictions.append(pred == np.argmax(y))   
        return np.mean(predictions)
def train(model, x_train, y_train, x_val, y_val, epochs=100, lr=0.0075):
    start_time = time.time()
    for iteration in range(epochs):
        for x,y in zip(x_train, y_train):
            output = model.L_model_forward(x)
            changes_to_w = model.L_model_backward(y, output)
            model.update_network_parameters(changes_to_w, lr)
        
        accuracy = compute_accuracy(model, x_val, y_val)
        print('Epoch: {0}, Time Spent: {1:.2f}s, Accuracy: {2:.2f}%'.format(
            iteration+1, time.time() - start_time, accuracy * 100
        ))

In [52]:
dnn = Model(x=784, h1=128, h2=64, y=10)
train(dnn, trainX, trainy, testX, testy)

W1 (128, 784)
Epoch: 1, Time Spent: 60.41s, Accuracy: 2.46%
Epoch: 2, Time Spent: 121.16s, Accuracy: 23.09%
Epoch: 3, Time Spent: 181.93s, Accuracy: 30.06%
Epoch: 4, Time Spent: 242.38s, Accuracy: 32.69%
Epoch: 5, Time Spent: 304.00s, Accuracy: 34.63%
Epoch: 6, Time Spent: 365.21s, Accuracy: 36.33%
Epoch: 7, Time Spent: 425.99s, Accuracy: 37.54%
Epoch: 8, Time Spent: 486.39s, Accuracy: 38.32%
Epoch: 9, Time Spent: 546.66s, Accuracy: 39.24%
Epoch: 10, Time Spent: 607.44s, Accuracy: 40.45%
Epoch: 11, Time Spent: 668.05s, Accuracy: 41.61%


KeyboardInterrupt: ignored

Epoch: 1, Time Spent: 60.41s, Accuracy: 2.46%
Epoch: 2, Time Spent: 121.16s, Accuracy: 23.09%
Epoch: 3, Time Spent: 181.93s, Accuracy: 30.06%
Epoch: 4, Time Spent: 242.38s, Accuracy: 32.69%
Epoch: 5, Time Spent: 304.00s, Accuracy: 34.63%
Epoch: 6, Time Spent: 365.21s, Accuracy: 36.33%
Epoch: 7, Time Spent: 425.99s, Accuracy: 37.54%
Epoch: 8, Time Spent: 486.39s, Accuracy: 38.32%
Epoch: 9, Time Spent: 546.66s, Accuracy: 39.24%
Epoch: 10, Time Spent: 607.44s, Accuracy: 40.45%
Epoch: 11, Time Spent: 668.05s, Accuracy: 41.61%